<a href="https://colab.research.google.com/github/Lajith-Puthuchery/Champions-League-Draw-Simulator/blob/main/CLDraw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Teams.csv to Teams (1).csv


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import random

In [ ]:
teams = pd.read_csv("Teams.csv")
teams.head()

,Team,Abbreviation,Group Winner,Group Name,Country
0,Manchester City,MCI,True,A,England
1,Manchester United,MUN,True,F,England
2,Liverpool,LIV,True,B,England
3,Bayern Munich,BAY,True,E,Germany
4,Real Madrid,RMA,True,D,Spain


In [ ]:
teams['Possible Opponents'] = [[]] * 16
teams['Number of possible opponents'] = [0] * 16
teams['Picked'] = [False] * 16
teams

,Team,Abbreviation,Group Winner,Group Name,Country,Possible Opponents,Number of possible opponents,Picked
0,Manchester City,MCI,True,A,England,[],0,False
1,Manchester United,MUN,True,F,England,[],0,False
2,Liverpool,LIV,True,B,England,[],0,False
3,Bayern Munich,BAY,True,E,Germany,[],0,False
4,Real Madrid,RMA,True,D,Spain,[],0,False
5,LOSC Lille,LIL,True,G,France,[],0,False
6,Ajax,AJA,True,C,Netherlands,[],0,False
7,Juventus,JUV,True,H,Italy,[],0,False
8,Paris Saint Germain,PAR,False,A,France,[],0,False
9,Villareal,VIL,False,F,Spain,[],0,False


In [ ]:
for i in range(16):
  teams['Possible Opponents'][i] = []
  for j in range(16):
    if(teams['Group Winner'][i] != teams['Group Winner'][j]) & (teams['Country'][i] != teams['Country'][j]) & (teams['Group Name'][i] != teams['Group Name'][j]):
      teams['Possible Opponents'][i].append(teams['Abbreviation'][j])
  teams['Number of possible opponents'][i] = len(teams['Possible Opponents'][i])

teams.head(16)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Team,Abbreviation,Group Winner,Group Name,Country,Possible Opponents,Number of possible opponents,Picked
0,Manchester City,MCI,True,A,England,"[VIL, ATL, BEN, INT, SAL, LIS]",6,False
1,Manchester United,MUN,True,F,England,"[PAR, ATL, BEN, INT, SAL, LIS]",6,False
2,Liverpool,LIV,True,B,England,"[PAR, VIL, BEN, INT, SAL, LIS]",6,False
3,Bayern Munich,BAY,True,E,Germany,"[PAR, VIL, ATL, INT, SAL, LIS, CHE]",7,False
4,Real Madrid,RMA,True,D,Spain,"[PAR, BEN, SAL, LIS, CHE]",5,False
5,LOSC Lille,LIL,True,G,France,"[VIL, ATL, BEN, INT, LIS, CHE]",6,False
6,Ajax,AJA,True,C,Netherlands,"[PAR, VIL, ATL, BEN, INT, SAL, CHE]",7,False
7,Juventus,JUV,True,H,Italy,"[PAR, VIL, ATL, BEN, SAL, LIS]",6,False
8,Paris Saint Germain,PAR,False,A,France,"[MUN, LIV, BAY, RMA, AJA, JUV]",6,False
9,Villareal,VIL,False,F,Spain,"[MCI, LIV, BAY, LIL, AJA, JUV]",6,False


In [ ]:
first_team = [''] * 8
second_team = [''] * 8
vs = ['vs'] * 8

fixtures = pd.DataFrame({'First Team':first_team, 'vs': vs, 'Second Team': second_team})
fixtures

,First Team,vs,Second Team
0,,vs,
1,,vs,
2,,vs,
3,,vs,
4,,vs,
5,,vs,
6,,vs,
7,,vs,


In [ ]:
def reset_team(teams, fixtures, m):
  teams = teams[teams['Picked']==False].reset_index(drop=True)
  if len(teams) is 0:
    return teams, fixtures, m

  for i in range(len(teams)):
    if fixtures['First Team'][m] in teams['Possible Opponents'][i]:
      teams['Possible Opponents'][i].remove(fixtures['First Team'][m])
    if fixtures['Second Team'][m] in teams['Possible Opponents'][i]:
      teams['Possible Opponents'][i].remove(fixtures['Second Team'][m])
    teams['Number of possible opponents'][i] = len(teams['Possible Opponents'][i])
  return teams, fixtures, m

In [ ]:
def pred_fixture(teams, fixtures, m):
  group_winners_left=len(teams[(teams['Group Winner']==True) & (teams['Picked']==False)])
  first=random.randint(0, group_winners_left - 1)
  print(str(first)+" Hiiiiiiiiiiiiiii")
  fixtures['First Team'][m] = teams['Abbreviation'][first]
  teams['Picked'][teams['Abbreviation']== fixtures['First Team'][m]] = True
  display(fixtures.loc[[m]])

  second=random.randint(0, teams['Number of possible opponents'][first]-1)
  fixtures['Second Team'][m] = teams['Possible Opponents'][first][second]

  teams['Picked'][teams['Abbreviation']==fixtures['Second Team'][m]] = True
  teams, fixtures, m = reset_team(teams, fixtures, m)
  display(fixtures.loc[[m]])
  m=m+1
  return teams, fixtures, m


In [ ]:
def check_implies(teams, fixtures, m):
  if min(teams['Number of possible opponents']) > 1:
    return teams,fixtures, m
  else:
    team_index = teams[teams['Number of possible opponents'] == 1].index[0]
    if teams['Group Winner'][team_index] == True:
      fixtures['First Team'][m] = teams['Abbreviation'][team_index]
      fixtures['Second Team'][m] = teams['Possible Opponents'][team_index][0]
    display(fixtures.loc[[m]])
    teams['Picked'][teams['Abbreviation']== fixtures['First Team'][m]] = True
    teams['Picked'][teams['Abbreviation']==fixtures['Second Team'][m]] = True
    teams, fxitures, m = reset_team(teams, fixtures, m)
    m=m+1
    return teams, fixtures, m

In [ ]:
m=0
while len(teams) is not 0:
  if len(teams) is 2:
    teams, fixtures, m = check_implies(teams, fixtures, m)
    continue
  teams, fixtures, m = pred_fixture(teams,fixtures, m)
  teams, fixtures, m = check_implies(teams,fixtures,m)

1 Hiiiiiiiiiiiiiii


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,First Team,vs,Second Team
0,MUN,vs,


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,First Team,vs,Second Team
0,MUN,vs,BEN


...

,First Team,vs,Second Team
6,BAY,vs,


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,First Team,vs,Second Team
6,BAY,vs,SAL


,First Team,vs,Second Team
7,MCI,vs,ATL


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
teams

,Team,Abbreviation,Group Winner,Group Name,Country,Possible Opponents,Number of possible opponents,Picked


In [ ]:
fixtures

,First Team,vs,Second Team
0,MUN,vs,BEN
1,JUV,vs,PAR
2,LIV,vs,INT
3,AJA,vs,CHE
4,RMA,vs,LIS
5,LIL,vs,VIL
6,BAY,vs,SAL
7,MCI,vs,ATL


In [ ]:
print(teams.columns.tolist())

['Team', 'Abbreviation', 'Group Winner', 'Group Name', 'Country', 'Possible Opponents', 'Number of possible opponents', 'Picked']
